# OMOP/N3C Examples

See [https://github.com/jhu-bids/TermHub/issues/516](https://github.com/jhu-bids/TermHub/issues/516)


## Basic term lookup

First we will create an adapter object for a previously created sqlite database containing the N3C version of OMOP.

In [88]:
from oaklib import get_adapter
adapter = get_adapter('input/n3c.db')

Next we will do a lookup -- note in OAK all IDs are compact URIs (prefixed identifiers)

In [89]:
TERM_ID = "omop:4195673"
print(adapter.label(TERM_ID))

Angioplasty of posterior tibial artery


If you have a list of ids you can use the `labels` method:

In [90]:
for id, label in adapter.labels([TERM_ID, "omop:45933598"]):
    print(id, label)

omop:4195673 Angioplasty of posterior tibial artery
omop:45933598 Postsurgical Percutaneous Transluminal Coronary Angioplasty Status


## Basic Search

OAK has a number of different ways to search ontologies or to do boolean queries combining
lexical search and graph constraints.

We will start with a simple lookup-by-label, which considered the most basic search:

In [91]:
list(adapter.basic_search("Angioplasty"))

['omop:1018433']

Next we will do a partial string match search -- for this we will need a SearchConfiguration object:

In [93]:
from oaklib.datamodels.search import SearchConfiguration
for t in list(adapter.basic_search("Angioplasty", SearchConfiguration(is_partial=True)))[0:5]:
    print(t, adapter.label(t))

omop:45933598 Postsurgical Percutaneous Transluminal Coronary Angioplasty Status
omop:1389714 Percutaneous transluminal angioplasty of native or recurrent coarctation of the aorta
omop:2101787 Anesthesia for angioplasty (Deprecated)
omop:2107779 Transluminal balloon angioplasty, open; renal or other visceral artery
omop:2107780 Transluminal balloon angioplasty, open; aortic


Note we truncated the results to 5 for brevity.

Lexical search is currently quite slow for the sqlite backend. In future it will be possible
to use a hybrid approach with lucene-backed search using Solr.

## Command Line Simple Search

We can do the same thing on the command line:

In [94]:
!runoak -i $db/omop.db info l~Angioplasty | head -5

omop:45933598 ! Postsurgical Percutaneous Transluminal Coronary Angioplasty Status
omop:1389714 ! Percutaneous transluminal angioplasty of native or recurrent coarctation of the aorta
omop:2101787 ! Anesthesia for angioplasty (Deprecated)
omop:2107779 ! Transluminal balloon angioplasty, open; renal or other visceral artery
omop:2107780 ! Transluminal balloon angioplasty, open; aortic


## Graph Queries

Ontologies and ontology-like structures can be projected onto graphs, and OAK provides a number of
graph-oriented queries. For now we only consider IS_A graphs.

In [95]:
from oaklib.datamodels.vocabulary import IS_A
ancs = list(adapter.ancestors([TERM_ID], predicates=[IS_A]))
for a in ancs:
    print(a, adapter.label(a))

omop:4195673 Angioplasty of posterior tibial artery
omop:4000756 Leg repair
omop:4002031 Cardiovascular system repair
omop:4012185 Cardiovascular surgical procedure
omop:4030028 Surgical procedure on lower extremity
omop:4050128 Angioplasty of artery
omop:4050134 Angioplasty of crural artery
omop:4054559 Repair of blood vessel
omop:4062347 Surgical repair of artery of extremity
omop:4091623 Surgical repair of lower extremity
omop:4148948 Vascular surgery procedure
omop:4159949 Surgical procedure on soft tissue
omop:4160912 Vascular surgical procedure on lower limb
omop:4177089 Surgical repair procedure by device
omop:4181193 Limb operation
omop:4181322 Surgical repair procedure by body site
omop:4184453 Operative procedure on lower leg
omop:4185115 Surgical repair
omop:4190070 Angioplasty of artery of lower extremity
omop:4195673 Angioplasty of posterior tibial artery
omop:4301351 Surgical procedure
omop:4302652 Angioplasty of blood vessel
omop:4311041 Repair of artery
omop:4324523 Dil

In [96]:
ancs = list(adapter.ancestors([TERM_ID], predicates=[IS_A]))
for a in ancs:
    for _, p, o in adapter.relationships([a], predicates=[IS_A]):
        print(p, o, adapter.label(o))

rdfs:subClassOf omop:4000756 Leg repair
rdfs:subClassOf omop:4050134 Angioplasty of crural artery
rdfs:subClassOf omop:4091623 Surgical repair of lower extremity
rdfs:subClassOf omop:4184453 Operative procedure on lower leg
rdfs:subClassOf omop:4012185 Cardiovascular surgical procedure
rdfs:subClassOf omop:4181322 Surgical repair procedure by body site
rdfs:subClassOf omop:4301351 Surgical procedure
rdfs:subClassOf omop:4181193 Limb operation
rdfs:subClassOf omop:4302652 Angioplasty of blood vessel
rdfs:subClassOf omop:4311041 Repair of artery
rdfs:subClassOf omop:4190070 Angioplasty of artery of lower extremity
rdfs:subClassOf omop:4002031 Cardiovascular system repair
rdfs:subClassOf omop:4148948 Vascular surgery procedure
rdfs:subClassOf omop:4311041 Repair of artery
rdfs:subClassOf omop:4331725 Operative procedure on artery of extremity
rdfs:subClassOf omop:4030028 Surgical procedure on lower extremity
rdfs:subClassOf omop:4181322 Surgical repair procedure by body site
rdfs:subClass

In [97]:
descs = list(adapter.descendants(["omop:4181322"], predicates=[IS_A]))
for d in descs[0:5]:
    print(d, adapter.label(d))

omop:3183297 Transvaginal urethral sling repair
omop:2730516 Bypass Right Femoral Artery to Posterior Tibial Artery, Percutaneous Endoscopic Approach
omop:4210771 Reconstruction of eyelid, full-thickness
omop:2733612 Extraction of Left Brachial Vein, Percutaneous Approach
omop:2725058 Bypass Right Ventricle to Right Pulmonary Artery with Synthetic Substitute, Percutaneous Endoscopic Approach


## Simple Pairwise Semantic Similarity

In [98]:
from linkml_runtime.dumpers import yaml_dumper
print(yaml_dumper.dumps(adapter.pairwise_similarity(TERM_ID, "omop:2733612")))

subject_id: omop:4195673
object_id: omop:2733612
ancestor_id: omop:4302652
ancestor_information_content: 10.432664389401875
jaccard_similarity: 0.3793103448275862
phenodigm_score: 1.9892756287187816


## Paths

In [99]:
for path in adapter.paths([TERM_ID], ["omop:4210771"], predicates=[IS_A]):
    print([(elt, adapter.label(elt)) for elt in path])

[('omop:4195673', 'Angioplasty of posterior tibial artery'), ('omop:4210771', 'Reconstruction of eyelid, full-thickness'), ('omop:4195673', 'Angioplasty of posterior tibial artery')]
[('omop:4195673', 'Angioplasty of posterior tibial artery'), ('omop:4210771', 'Reconstruction of eyelid, full-thickness'), ('omop:4000756', 'Leg repair')]
[('omop:4195673', 'Angioplasty of posterior tibial artery'), ('omop:4210771', 'Reconstruction of eyelid, full-thickness'), ('omop:4091623', 'Surgical repair of lower extremity')]
[('omop:4195673', 'Angioplasty of posterior tibial artery'), ('omop:4210771', 'Reconstruction of eyelid, full-thickness'), ('omop:4181322', 'Surgical repair procedure by body site')]
[('omop:4195673', 'Angioplasty of posterior tibial artery'), ('omop:4210771', 'Reconstruction of eyelid, full-thickness'), ('omop:4161058', 'Surgical repair of head and neck structure')]
[('omop:4195673', 'Angioplasty of posterior tibial artery'), ('omop:4210771', 'Reconstruction of eyelid, full-thi

## Subgraphs

In [100]:
seeds = [TERM_ID, "omop:4210771", "omop:2733612"]
g = adapter.ancestor_graph(seeds, predicates=[IS_A])

In [101]:
from oaklib.utilities.obograph_utils import graph_to_image
graph_to_image(g, seeds=seeds, imgfile="output/angioplasty.png", format="png")

![img](output/angioplasty.png)

In [102]:
from oaklib.utilities.obograph_utils import default_stylemap_path

stylemap = default_stylemap_path()
graph_to_image(g, seeds=seeds, imgfile="output/angioplasty-styled.png", format="png", stylemap=stylemap)

![img](output/angioplasty-styled.png)

In [36]:
for s, p, o in adapter.gap_fill_relationships(["omop:4195673", "omop:4210771", 
                                               "omop:2733612", "omop:4054559", "omop:4301351"],
                                                predicates=[IS_A]):
    print(s, adapter.label(s), "==>", o, adapter.label(o))

omop:2733612 Extraction of Left Brachial Vein, Percutaneous Approach ==> omop:4054559 Repair of blood vessel
omop:4054559 Repair of blood vessel ==> omop:4301351 Surgical procedure
omop:4195673 Angioplasty of posterior tibial artery ==> omop:4054559 Repair of blood vessel
omop:4210771 Reconstruction of eyelid, full-thickness ==> omop:4301351 Surgical procedure


## Export to networkx

In [37]:
from oaklib.utilities.obograph_utils import as_multi_digraph
nx_g = as_multi_digraph(g)

In [38]:
nx_g.nodes

NodeView(('omop:4027561', 'omop:2733612', 'omop:40489873', 'omop:4177089', 'omop:4302652', 'omop:4054559', 'omop:4324523', 'omop:4301351', 'omop:4002031', 'omop:4148948', 'omop:4012185', 'omop:4159949', 'omop:4181322', 'omop:4185115', 'omop:4134598', 'omop:4330850', 'omop:4210771', 'omop:4045162', 'omop:4161828', 'omop:4161058', 'omop:4249123', 'omop:4139008', 'omop:4031321', 'omop:4154279', 'omop:4233946', 'omop:4040721', 'omop:4000756', 'omop:4195673', 'omop:4050134', 'omop:4190070', 'omop:4050128', 'omop:4062347', 'omop:4091623', 'omop:4160912', 'omop:46271049', 'omop:4030028', 'omop:4181193', 'omop:4311041', 'omop:4331725', 'omop:4184453'))

In [39]:
# find graph statistics using networkx
import networkx as nx
nx.degree_centrality(nx_g)

{'omop:4027561': 0.05128205128205128,
 'omop:2733612': 0.05128205128205128,
 'omop:40489873': 0.07692307692307693,
 'omop:4177089': 0.07692307692307693,
 'omop:4302652': 0.10256410256410256,
 'omop:4054559': 0.10256410256410256,
 'omop:4324523': 0.07692307692307693,
 'omop:4301351': 0.1794871794871795,
 'omop:4002031': 0.10256410256410256,
 'omop:4148948': 0.1282051282051282,
 'omop:4012185': 0.07692307692307693,
 'omop:4159949': 0.05128205128205128,
 'omop:4181322': 0.10256410256410256,
 'omop:4185115': 0.10256410256410256,
 'omop:4134598': 0.05128205128205128,
 'omop:4330850': 0.07692307692307693,
 'omop:4210771': 0.02564102564102564,
 'omop:4045162': 0.05128205128205128,
 'omop:4161828': 0.07692307692307693,
 'omop:4161058': 0.07692307692307693,
 'omop:4249123': 0.05128205128205128,
 'omop:4139008': 0.07692307692307693,
 'omop:4031321': 0.05128205128205128,
 'omop:4154279': 0.05128205128205128,
 'omop:4233946': 0.07692307692307693,
 'omop:4040721': 0.02564102564102564,
 'omop:400075

## Term metadata

In [40]:
adapter.entity_metadata_map(TERM_ID)

{'id': ['omop:4195673'],
 'omop:concept_class_id': ['Procedure'],
 'omop:concept_code': ['312644004'],
 'omop:domain_id': ['Procedure'],
 'omop:standard_concept': ['S'],
 'omop:valid_end_date': ['2099-12-31'],
 'omop:valid_start_date': ['2002-01-31'],
 'omop:vocabulary_id': ['SNOMED'],
 'rdfs:label': ['Angioplasty of posterior tibial artery'],
 'sh:prefix': ['omop'],
 'schema:url': ['https://athena.ohdsi.org/search-terms/terms/4195673'],
 'rdfs:isDefinedBy': ['https://athena.ohdsi.org/search-terms/terms/']}

In [41]:
adapter.label("omop:312644004")

## Semantic Similarity using Rust

In [42]:
simadapter = get_adapter('semsimian:input/n3c.db')

In [43]:
terms1 = ["omop:4195673", "omop:4000756", "omop:4002031", "omop:4012185"]
terms2 = ["omop:4030028", "omop:4050128", "omop:4050134", "omop:4054559"]


In [44]:
tsps = simadapter.termset_pairwise_similarity(terms1, terms2, predicates=[IS_A])

In [45]:
print(yaml_dumper.dumps(tsps))

subject_termset:
  omop:4000756:
    id: omop:4000756
    label: Leg repair
  omop:4002031:
    id: omop:4002031
    label: Cardiovascular system repair
  omop:4012185:
    id: omop:4012185
    label: Cardiovascular surgical procedure
  omop:4195673:
    id: omop:4195673
    label: Angioplasty of posterior tibial artery
object_termset:
  omop:4050128:
    id: omop:4050128
    label: Angioplasty of artery
  omop:4054559:
    id: omop:4054559
    label: Repair of blood vessel
  omop:4030028:
    id: omop:4030028
    label: Surgical procedure on lower extremity
  omop:4050134:
    id: omop:4050134
    label: Angioplasty of crural artery
subject_best_matches:
  omop:4000756:
    match_source: omop:4000756
    score: 10.984781775651545
    similarity:
      subject_id: omop:4000756
      object_id: omop:4050134
      ancestor_id: omop:4091623
      ancestor_label: ''
      ancestor_information_content: 10.984781775651545
      jaccard_similarity: 0.25
      phenodigm_score: 1.65716488132982

## Mondo Mappings

We will now demonstrate mapping to Mondo. Mondo has mappings to many sources - but not OMOP.

The n3c OMOP does however have mappings to SNOMED (structured in an unusual way in the OWL),
so we can use these as a join point to Mondo

In [49]:
PD = "omop:381270"
print(adapter.label(PD))

Parkinson's disease


In [55]:
def omop_mappings(id):
    # we SHOULD be able to just do this:
    # adapter.mappings([id], ...)
    # however, mappings are stored in the upstream OWL in a non-standard way
    return [f"SCTID:{code}" for code in adapter.entity_metadata_map(id).get("omop:concept_code", [])]

print(omop_mappings(PD))
    

['SCTID:49049000']


In [56]:
mondo = get_adapter("sqlite:obo:mondo")

In [60]:
for m in mondo.sssom_mappings(omop_mappings(PD)):
    print(m.subject_id,m.predicate_id, m.object_id, PD)

MONDO:0005180 oio:hasDbXref SCTID:49049000 omop:381270


In [65]:
def transitive_mappings(omop_id):
    for m in mondo.sssom_mappings(omop_mappings(omop_id)):
        for m2 in mondo.sssom_mappings([m.subject_id]):
            yield omop_id, m.subject_id, m.object_id, m2.object_id

In [66]:
for m in transitive_mappings(PD):
    print(m)

('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'DOID:14330')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'EFO:0002508')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'ICD9:332')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'ICD9:332.0')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'MESH:D010300')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'NCIT:C26845')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'NIFSTD:birnlex_2098')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'OMIMPS:168600')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'Orphanet:319705')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'SCTID:49049000')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'UMLS:C0030567')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', '<https://omim.org/phenotypicSeries/PS168600>')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'NCIT:C26845')
('omop:381270', 'MONDO:0005180', 'SCTID:49049000', 'DOID:14330')
('omop:381270', 'MONDO:000518

## Value Set Expansion

In [80]:
!cat input/n3c_config.yaml

default_resolver:
  name: omop
  shorthand: sqlite:input/n3c.db


In [83]:
!cat input/n3c-example-intensional-value-set.yaml

id: https://w3id.org/linkml/examples/enums
title: Dynamic Enums Example
name: dynamicenums-example
description: This demonstrates the use of dynamic enums
license: https://creativecommons.org/publicdomain/zero/1.0/

prefixes:
  linkml: https://w3id.org/linkml/
  ex: https://w3id.org/linkml/examples/enums/
  sh: https://w3id.org/shacl/
  bioregistry: https://bioregistry.io/registry/
  MONDO: http://purl.obolibrary.org/obo/MONDO_
  omop: https://athena.ohdsi.org/search-terms/terms/
  loinc: http://loinc.org/

default_prefix: ex
default_range: string

imports:
  - linkml:types


enums:
  Synucleinopathies:
    reachable_from:
      include_self: true
      source_ontology: local:omop
      source_nodes:
        - omop:37203944


In [84]:
from oaklib.utilities.subsets.value_set_expander import ValueSetExpander

expander = ValueSetExpander()

In [85]:
from oaklib.datamodels.value_set_configuration import ValueSetConfiguration
from linkml_runtime.loaders import yaml_loader

expander.configuration = yaml_loader.load("input/n3c_config.yaml", target_class=ValueSetConfiguration)

In [86]:
expander.expand_in_place(
        schema_path="input/n3c-example-intensional-value-set.yaml", value_set_names=["Synucleinopathies"], 
        output_path="output/synucleinopathies.yaml"
    )

In [87]:
!cat output/synucleinopathies.yaml

id: https://w3id.org/linkml/examples/enums
title: Dynamic Enums Example
name: dynamicenums-example
description: This demonstrates the use of dynamic enums
license: https://creativecommons.org/publicdomain/zero/1.0/

prefixes:
  linkml: https://w3id.org/linkml/
  ex: https://w3id.org/linkml/examples/enums/
  sh: https://w3id.org/shacl/
  bioregistry: https://bioregistry.io/registry/
  MONDO: http://purl.obolibrary.org/obo/MONDO_
  omop: https://athena.ohdsi.org/search-terms/terms/
  loinc: http://loinc.org/

default_prefix: ex
default_range: string

imports:
- linkml:types


enums:
  Synucleinopathies:
    reachable_from:
      include_self: true
      source_ontology: local:omop
      source_nodes:
      - omop:37203944
    permissible_values:
      omop:44800441:
        text: omop:44800441
        description: '[X]Parkinsonism in diseases classified elsewhere'
        meaning: omop:44800441
      omop:380701:
        text: omop:380701
        description: Diffuse Lewy body disease
  

## Command Line Value Set Expansion

In [73]:
!vskit expand -c input/n3c_config.yaml -s input/n3c-example-intensional-value-set.yaml Synucleinopathies -o output/synucleinopathies.yaml